In [ ]:
# Preload VIIRS for Fast Windowed Reads

import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.windows import from_bounds
import numpy as np
import glob
import os
from shapely.geometry import box

TILE_DIR = "../data/spacenet/AOI_2_Vegas/PS-RGB/"
GEOJSON_DIR = "../data/spacenet/AOI_2_Vegas/geojson_buildings/"
OUTPUT_CSV = "../data/output/vegas_all_tile_metrics.csv"

results = []

for img_path in glob.glob(os.path.join(TILE_DIR, "SN2_buildings_train_AOI_2_Vegas_PS-RGB_img*.tif")):

    tile_id = os.path.basename(img_path).split("_img")[1].split(".")[0]

    # ---- Load GEOJSON (footprints) ----
    gj_path = os.path.join(GEOJSON_DIR, f"SN2_buildings_train_AOI_2_Vegas_geojson_buildings_img{tile_id}.geojson")
    if not os.path.exists(gj_path):
        continue

    gdf = gpd.read_file(gj_path)

    # --- Reproject shapes to UTM so area is correct ---
    gdf_utm = gdf.to_crs("EPSG:32611")

    building_count = len(gdf_utm)
    total_area = gdf_utm.area.sum()
    mean_area = gdf_utm.area.mean()

    # --- Load tile bounds (lat/lon) ---
    with rasterio.open(img_path) as src:
        b = src.bounds
        # convert to UTM11 bbox
        tile_poly = gpd.GeoSeries([box(b.left, b.bottom, b.right, b.top)], crs=src.crs)
        tile_poly_utm = tile_poly.to_crs("EPSG:32611")
        ub = tile_poly_utm.iloc[0].bounds

    # --- Crop VIIRS FAST using window ---
    try:
        win = from_bounds(*ub, transform=viirs_ds.transform)
        arr = viirs_ds.read(1, window=win)
        arr = np.where(arr < 0, 0, arr)
    except:
        arr = np.zeros((1, 1))

    radiance_mean = float(arr.mean())
    radiance_sum = float(arr.sum())
    radiance_nonzero = int((arr > 0).sum())

    # ---- Save metrics ----
    results.append({
        "tile": tile_id,
        "building_count": building_count,
        "building_total_area_m2": total_area,
        "building_mean_area_m2": mean_area,
        "lights_mean": radiance_mean,
        "lights_sum": radiance_sum,
        "lights_nonzero": radiance_nonzero
    })

df = pd.DataFrame(results)
df.to_csv(OUTPUT_CSV, index=False)
print("Saved:", OUTPUT_CSV)

In [ ]:
# STEP 2: Night-Lights Extraction (Fast, M3-optimized)

import pandas as pd
import rasterio
from rasterio.windows import from_bounds
import numpy as np
import os
from pyproj import Transformer

# ==========================================================
# CONFIG
# ==========================================================
INPUT_CSV = "../data/output/shanghai_all_tile_metrics.csv"
VIIRS_UTM51 = "../data/night_lights/viirs_global_utm51.tif"
OUTPUT_CSV = "../data/output/shanghai_with_lights.csv"

# Shanghai coordinate systems
WGS84 = "EPSG:4326"
UTM51 = "EPSG:32651"

# ==========================================================
# LOAD TILE METRICS (Step 1 results)
# ==========================================================
df = pd.read_csv(INPUT_CSV)
print("Tiles loaded:", len(df))

# ==========================================================
# PREPARE COORDINATE TRANSFORMER (WGS84 → UTM51)
# ==========================================================
transformer = Transformer.from_crs(WGS84, UTM51, always_xy=True)

# ==========================================================
# OPEN VIIRS (already in EPSG:32651)
# ==========================================================
with rasterio.open(VIIRS_UTM51) as src:
    print("VIIRS CRS:", src.crs)

    lights_mean = []
    lights_sum = []
    lights_nonzero = []

    for idx, row in df.iterrows():

        # ------------------------------------------
        # 1. Convert tile bounding box → UTM51
        # ------------------------------------------
        minx, miny = transformer.transform(row["minx"], row["miny"])
        maxx, maxy = transformer.transform(row["maxx"], row["maxy"])

        # ------------------------------------------
        # 2. Create raster window
        # ------------------------------------------
        try:
            window = from_bounds(minx, miny, maxx, maxy, src.transform)
            arr = src.read(1, window=window)

            if arr.size == 0:
                lights_mean.append(0)
                lights_sum.append(0)
                lights_nonzero.append(0)
                continue

            arr = arr.astype(float)
            arr[arr < 0] = 0  # remove negatives

            lights_mean.append(arr.mean())
            lights_sum.append(arr.sum())
            lights_nonzero.append(np.count_nonzero(arr))

        except Exception as e:
            print("Error on tile", row["tile_id"], ":", e)
            lights_mean.append(0)
            lights_sum.append(0)
            lights_nonzero.append(0)

# ==========================================================
# ADD LIGHT METRICS TO DATAFRAME
# ==========================================================
df["lights_mean"] = lights_mean
df["lights_sum"] = lights_sum
df["lights_nonzero"] = lights_nonzero

# ==========================================================
# SAVE OUTPUT
# ==========================================================
df.to_csv(OUTPUT_CSV, index=False)
print("DONE. Saved:", OUTPUT_CSV)

# Show sample
df.head()